In [1]:
from collections.abc import Iterable
from pprint import pprint

import pymongo
from collections import Counter
from itertools import chain

In [2]:
c = pymongo.MongoClient()

In [3]:
#db0 = c.neuro1337
db0 = c.test_consensus
db1 = c.neuro1338
db2 = c.neuro1339

In [6]:
def coolify(obj):
    if type(obj) is dict:
        return CoolDict(obj)
    elif type(obj) is list:
        return CoolList(obj)
    else:
        return obj

    
class CoolDict(dict):
    def __getattr__(self, name):
        attr = self[name]
        return coolify(attr)
    
class CoolList(list):
    def __getitem__(self, i):
        item = super().__getitem__(i)
        return coolify(item)

    def __getattr__(self, name):
        return CoolList(getattr(x, name) for x in self)
    
    @property
    def flatten(self):
        return CoolList(chain(*self))
    
    def __iter__(self):
        return (coolify(obj) for obj in super().__iter__())


class Block(CoolDict):
    def __init__(self, arg, db=db0):
        self.db = db
        if type(arg) == dict:
            super().__init__(arg)
        elif type(arg) == str:
            super().__init__(db.blocks.find_one({"block.header.id.data": arg}))
        elif type(arg) == int:
            super().__init__(db.blocks.find_one({"block.header.height": arg}))

    @property
    def previousAssembly(self):
        return Assembly(self.previousAssemblyId.data, db=self.db)
    
    @property
    def previousBlock(self):
        return Block(self.block.header.previousBlockHash.data, db=self.db)

    @property
    def transactions(self):
        result = []
        for t in self.db.transactions.find(
            {"blockId.data": self.block.header.id.data, "isCoinbase": False}
        ):
            result.append(Transaction(t))
        return result

    @property
    def coinbase(self):
        return Transaction(
            self.db.transactions.find_one(
                {"blockId.data": self.block.header.id.data, "isCoinbase": True}
            )
        )


class Assembly(CoolDict):
    def __init__(self, arg, db=db0):
        self.db = db
        if type(arg) == dict:
            super().__init__(arg)
        elif type(arg) == str:
            super().__init__(db.assemblies.find_one({"id.data": arg}))
        elif type(arg) == int:
            super().__init__(db.assemblies.find_one({"height": arg}))

    @property
    def previousAssembly(self):
        return Assembly(self.previousAssemblyId.data, db=self.db)
    
    @property
    def piis(self):
        return CoolList([CoolDict(p) for p in self.db.pii.find({"assemblyId.data": self.id.data})])
    
    @property
    def blocks(self):
        return Blocks({"previousAssemblyId.data": self.previousAssemblyId.data})

    
class Transaction(CoolDict):
    def __init__(self, arg, db=db0, pool=True):
        self.db = db
        if type(arg) == dict:
            super().__init__(arg)
        else:
            query = {"transaction.id.data": arg}
            if not pool:
                query["blockId"] = {"$exists": True}
            super().__init__(db.transactions.find_one(query))

    @property
    def block(self):
        return Block(self.blockId.data, db=self.db)


class Transactions(CoolList):
    def __init__(self, arg=None, db=db0, pool=True):
        self.db = db
        if type(arg) is str:
            query = {"transaction.id.data": arg}
            if not pool:
                query["blockId"] = {"$exists": True}
        elif type(arg) is dict:
            query = arg
        elif isinstance(arg, Iterable):
            return super().__init__(arg)
        else:
            query = {}
        transactions = (Transaction(t) for t in db.transactions.find(query))
        super().__init__(transactions)

    @classmethod
    def by_input(cls, transaction_id, output_id, pool=True):
        query = {
            "transaction.inputs": {
                "$elemMatch": {
                    "id.data": transaction_id,
                    "outputId": output_id,
                }
            }
        }
        if not pool:
            query["blockId"] = {"$exists": True}
        return cls(query)
    
    @classmethod
    def by_input_address(cls, address):
        input_transactions = Transactions({"transaction.outputs.address.data": address})
        transactions = Transactions([])
        for input_transaction in input_transactions:
            for i, output_address in enumerate(input_transaction.transaction.outputs.address.data):
                if output_address == address:
                    output_id = i
                    break
            else:
                assert False
            transactions.extend(cls.by_input(input_transaction.transaction.id.data, output_id))
        return transactions
    
    @classmethod
    def incoming(cls, address, assembly):
        outgoing = cls.outgoing(address, assembly=None)
        outgoing_ids = set(outgoing.transaction.id.data)
        transactions = Transactions({"transaction.outputs.address.data": address})
        return Transactions(t for t in transactions 
                        if t.block.previousAssemblyId.data == assembly.previousAssemblyId.data
                            and t.transaction.id.data not in outgoing_ids
                            and not t.isCoinbase
                           ).deduplicate
    
    @classmethod
    def outgoing(cls, address, assembly):
        transactions = Transactions.by_input_address(address).deduplicate
        if assembly is None:
            return transactions
        return Transactions(t for t in transactions 
                        if t.block.previousAssemblyId.data == assembly.previousAssemblyId.data
                           and set(t.transaction.outputs.address.data) != set([address])
                           and not t.isCoinbase)
    
    @property
    def deduplicate(self):
        return Transactions({t.transaction.id.data: t for t in self}.values())

    
class Blocks(CoolList):
    def __init__(self, arg=None, db=db0):
        self.db = db
        if isinstance(arg, Iterable) and not type(arg) == dict:
            super().__init__(arg)
        else:
            if type(arg) == str:
                query = {"block.header.id.data": arg}
            elif type(arg) == int:
                query = {"block.header.height": arg}
            elif type(arg) == dict:
                query = arg
            else:
                query = {}
            super().__init__([Block(b) for b in db.blocks.find(query)])
    
    @classmethod
    def get_tip(cls, db=db0):
        return Block(db.blocks.find().sort("score", pymongo.DESCENDING).next())

    
class Assemblies(CoolList):
    def __init__(self, arg=None, db=db0):
        self.db = db
        if isinstance(arg, Iterable):
            super().__init__(arg)
        else:
            if type(arg) == int:
                query = {"height": arg}
            else:
                query = {}
        super().__init__([Assembly(a) for a in db.assemblies.find(query)])

In [10]:
Blocks().coinbase

[{'_id': ObjectId('5d090ccf85699a562f1a7c03'),
  'transaction': {'id': {'type': 'SHA256',
    'data': '1foQLRGLr6BHKPn9Av9dnQWtJ8V0H+EWfhGv6trOmsc='},
   'outputs': [{'address': {'data': 'N3WSXu5dzUkuj6UCzsUkzXsDt2rUm74pua'},
     'value': {'value': '100000'},
     'data': ''},
    {'address': {'data': 'N9E41Gu7BZy1SSnG3oqnRWBmHtjGYbBSd3'},
     'value': {'value': '100000'},
     'data': ''},
    {'address': {'data': 'NBXPijHYP2uobDegn1ncbQe5m2bJEDHViD'},
     'value': {'value': '100000'},
     'data': ''},
    {'address': {'data': 'ND8q7csEVi2XuPN7X28qWJqiwZhjyn7Hiu'},
     'value': {'value': '100000'},
     'data': ''}],
   'coinbaseHeight': 0},
  'isCoinbase': True,
  'blockId': {'type': 'SHA256',
   'data': 'jntmkDYVXWmhCOiiTwF0m7pWWPkD1ef6thlQzNMp5Qo='}},
 {'_id': ObjectId('5d090ccf85699a562f1a7c19'),
  'transaction': {'id': {'type': 'SHA256',
    'data': 'SQPW3HfPrZCmoeLYN7Gfbnquf3LR2L+2+iuy0cD1lpQ='},
   'outputs': [{'address': {'data': 'N9E41Gu7BZy1SSnG3oqnRWBmHtjGYbBSd3'},
   

In [5]:
assembly = Assembly(0)
assembly

TypeError: 'NoneType' object is not iterable

In [243]:
assembly.piis

[{'_id': ObjectId('5d08b77f85699a7eae30a531'),
  'address': {'data': 'N65B2ySQH88MNbsasubJd1pgtd6QHK6kT1'},
  'assemblyId': {'type': 'SHA256',
   'data': '53F8mcT75unNh+FcUOPeg8VXzEldvzhQLBBZmKWQR6E='},
  'score': '27.738556714491612',
  'rank': 0},
 {'_id': ObjectId('5d08b77f85699a7eae30a532'),
  'address': {'data': 'N83Zi8eFaKQF2tHzUmoZHaFwGKU8UT6WtM'},
  'assemblyId': {'type': 'SHA256',
   'data': '53F8mcT75unNh+FcUOPeg8VXzEldvzhQLBBZmKWQR6E='},
  'score': '27.453930706008748',
  'rank': 1},
 {'_id': ObjectId('5d08b77f85699a7eae30a533'),
  'address': {'data': 'N2L7ZWvmtFmPWUFKJaW9Lqg4WFBTutze9L'},
  'assemblyId': {'type': 'SHA256',
   'data': '53F8mcT75unNh+FcUOPeg8VXzEldvzhQLBBZmKWQR6E='},
  'score': '24.326021393899406',
  'rank': 2},
 {'_id': ObjectId('5d08b77f85699a7eae30a534'),
  'address': {'data': 'NDi1CPcircYgSRcSGtrdZVxnBGNwBrDJWM'},
  'assemblyId': {'type': 'SHA256',
   'data': '53F8mcT75unNh+FcUOPeg8VXzEldvzhQLBBZmKWQR6E='},
  'score': '21.489202644476691',
  'rank': 3}]

In [244]:
for address in Assembly(0).piis.address.data:
    print("address", address)
    print("incoming", len(Transactions.incoming(address, Assembly(0))))
    print("outgoing", len(Transactions.outgoing(address, Assembly(0))))

address N65B2ySQH88MNbsasubJd1pgtd6QHK6kT1
incoming 8
outgoing 8
address N83Zi8eFaKQF2tHzUmoZHaFwGKU8UT6WtM
incoming 7
outgoing 9
address N2L7ZWvmtFmPWUFKJaW9Lqg4WFBTutze9L
incoming 7
outgoing 6
address NDi1CPcircYgSRcSGtrdZVxnBGNwBrDJWM
incoming 10
outgoing 9


In [224]:
Block("LvWM3ljpfOrndycw7ads500Z3qz08TakkJBN7fLbLTA=").coinbase

{'_id': ObjectId('5d08b77785699a7eae30a4b3'),
 'transaction': {'id': {'type': 'SHA256',
   'data': 'px05qAL6684231rjccQdXKpAz4jIXrU3ehY2mYVQEK0='},
  'outputs': [{'address': {'data': 'N2L7ZWvmtFmPWUFKJaW9Lqg4WFBTutze9L'},
    'value': {'value': '100000'},
    'data': ''},
   {'address': {'data': 'N65B2ySQH88MNbsasubJd1pgtd6QHK6kT1'},
    'value': {'value': '100000'},
    'data': ''},
   {'address': {'data': 'NDi1CPcircYgSRcSGtrdZVxnBGNwBrDJWM'},
    'value': {'value': '100000'},
    'data': ''},
   {'address': {'data': 'N83Zi8eFaKQF2tHzUmoZHaFwGKU8UT6WtM'},
    'value': {'value': '100000'},
    'data': ''}],
  'coinbaseHeight': 0},
 'isCoinbase': True,
 'blockId': {'type': 'SHA256',
  'data': 'LvWM3ljpfOrndycw7ads500Z3qz08TakkJBN7fLbLTA='}}

In [ ]:
address = "NXAC8ttVhYj2s9RusKFdEwJk4Taz9F5k6b"

In [202]:
len(Transactions.incoming(address, Assembly(0)))

11

In [207]:
len(Transactions.incoming(address, Assembly(0)))

0

In [208]:
Assembly(0)

{'_id': ObjectId('5d08b16585699a788421cb1c'),
 'id': {'type': 'SHA256',
  'data': 'QmMQkM6WAm/AS9x5haoiEQUiNNkcIgFByXrhyj7NWLY='},
 'finishedComputation': True,
 'height': 0,
 'previousAssemblyId': {'type': 'SHA256',
  'data': '0WkRgtvrZEI0y2G4KmkRM+WiK0fb8oG0+U1jWT9UBMU='},
 'seed': 1,
 'nbAddresses': 4}

In [216]:
assembly = Assembly(0)
block = Block(assembly.id.data)
while True:
    print(block.block.header.id.data)
    #pprint(block)
    block = block.previousBlock
    if block.block.header.id.data == assembly.previousAssemblyId.data:
        break
    

QmMQkM6WAm/AS9x5haoiEQUiNNkcIgFByXrhyj7NWLY=
vmXh1sdTriPu+xIymUqZ6gb1wTmSCXBY6qgIVuDGQlw=
M2w3+AzI7X9J1aX75JynjCk62ID6+wXd+su9o+qszSY=
2lMeUwKBxDxXXeL9VYN79bJMLbxTwn8nZ4pkIpTYc1Q=
eL/J+1yZ/35Mi/7sOa95ngrP325xYMllndvTK1+vRFU=


TypeError: 'NoneType' object is not iterable

In [149]:
incoming = Transactions({"transaction.outputs.address.data": address})
incoming.block

[{'_id': ObjectId('5d08992e85699a622d7d74f2'),
  'block': {'header': {'id': {'type': 'SHA256',
     'data': 'KmbcH9HqhdgLMqGnmtAPP1EsJGDR1U6mtQF+XCQvhwM='},
    'timestamp': {'data': 1560744589},
    'previousBlockHash': {'type': 'SHA256', 'data': ''},
    'author': {'signature': {'type': 'SHA256',
      'data': 'zoulzXNmqnULTeRSs1CeSAZMbiA3XUFPltrZZ7azbRNiIspeyk+gexixa5vpDKauWM4pw+Eigszl56PKymPwew=='},
     'keyPub': {'type': 'ECP256K1',
      'rawData': 'AliY3WdS0eWj4YQr/gmrwBQuFrQnhb9xZIEXQ/zHi9AJ'}},
    'height': 0}},
  'branch': 'MAIN',
  'branchPath': {'branchIds': [0], 'blockNumbers': [0]},
  'score': 0,
  'previousAssemblyId': {'type': 'SHA256',
   'data': 'lgz/XP4TuRFyrVhwf5sXdL5H02yo3eQXrKTeMvfJdqA='}},
 {'_id': ObjectId('5d08992e85699a622d7d7508'),
  'block': {'header': {'id': {'type': 'SHA256',
     'data': 'Da+7/iM1iJQlxcww4/G2lupV6L3B/xBVIqdYTZEJPcA='},
    'timestamp': {'data': 1560744590},
    'previousBlockHash': {'type': 'SHA256',
     'data': 'KmbcH9HqhdgLMqGnmtAPP1

In [140]:
incoming = set(Transactions({"transaction.outputs.address.data": address}).transaction.id.data)
outgoing = set(Transactions.by_input_address(address).transaction.id.data)

In [141]:
len(outgoing)

13

In [142]:
len(incoming - outgoing)

14

In [128]:
Assembly(0).blocks.transactions.flatten

[{'_id': ObjectId('5d07ad3a85699a07cb1be3c9'),
  'transaction': {'id': {'type': 'SHA256',
    'data': 'CaYonAEzeI3EUnKGHPdGbpaE2pjEOehBFbQu8xYeon0='},
   'inputs': [{'id': {'type': 'SHA256',
      'data': 'anZPdncHJ4QQpWIBKQ3RWfx3cw70Ab7cN9onMb2z1Nc='},
     'outputId': 0,
     'signatureId': 0}],
   'outputs': [{'address': {'data': 'NC3bEbzgxVHjFA6hedTUTPREy8uxML4FJU'},
     'value': {'value': '50000'}},
    {'address': {'data': 'NBe5F9NeEq6yGAfhoM7LYucYQDouEZ4DwJ'},
     'value': {'value': '50000'}}],
   'signatures': [{'signature': {'type': 'SHA256',
      'data': 'RV8mFBJ0zNQVLmhmne6iMrBXrI8szUDQhZm7W0mbXH4F6bgfh25a/owUubklvxh62zc7HwtIIqWqqcaZNm82qA=='},
     'keyPub': {'type': 'ECP256K1',
      'rawData': 'AzpO8wptIiW2/juz1y43zlPMUMwUiNHkGW7R1JBtFa9i'}}]},
  'isCoinbase': False,
  'blockId': {'type': 'SHA256',
   'data': 'y371T6f5aw25lhRGb+Ch5aUKE/Xyiy/s0zPkGRuXX60='}},
 {'_id': ObjectId('5d07ad3a85699a07cb1be3ca'),
  'transaction': {'id': {'type': 'SHA256',
    'data': 'LUe0iGEdg

In [163]:
Blocks().transactions

[[],
 [{'_id': ObjectId('5d00f8d485699a6b9f147f19'),
   'transaction': {'id': {'type': 'SHA256',
     'data': '5w3fW6jD+MxQByGlPU0WLeDIp0dURmhNF3a890iAZ3k='},
    'inputs': [{'id': {'type': 'SHA256',
       'data': 'DjwN5LiQC82UShe//k8E3GPuL1zPGsmCfekbMsM2nG0='},
      'outputId': 1,
      'signatureId': 0},
     {'id': {'type': 'SHA256',
       'data': '8q0WUvewioqeundRz4VwquOxZLjVDCSBl2O8FB6qDe8='},
      'outputId': 0,
      'signatureId': 0},
     {'id': {'type': 'SHA256',
       'data': 'uwtPDPDpkF+LPyBgkK0B3MH+D/CHGVqALhIWwJNRrIY='},
      'outputId': 0,
      'signatureId': 0},
     {'id': {'type': 'SHA256',
       'data': 'YPiYW+RFGyG5z2tmWeD1/PXphIHhncffpnq9S2btcec='},
      'outputId': 0,
      'signatureId': 0}],
    'outputs': [{'address': {'data': 'N4QpgTY5AK2BSenTaZtmeoiBtr8nzZ6Ysb'},
      'value': {'value': '106250'}},
     {'address': {'data': 'NHk7xr8Ufg3M4XMS2bYmjMdiBxYrt4vW9L'},
      'value': {'value': '106250'}}],
    'signatures': [{'signature': {'type': 'SHA256'

In [155]:
Counter(Blocks().block.header.author.keyPub.rawData)

Counter({'AjlUcbknbTmHKPgriipCcxKr3WoLutOQfmnH+DCzt+aH': 2,
         'A39X2AdASSSRD0/g/Vq93r/zcmRaPTm/AOmZLRUPk4JT': 2,
         'AwGkuo9bw0c7CdtuADRYgBj5xuXvWYvuccjOoNSSihCs': 1,
         'ApZQs9+L8HFZj+pSHiJb2BEjXEWa3d9KJd39pa05iqSY': 1})

In [156]:
Counter(Blocks().block.header.author.keyPub.rawData)

Counter({'AjlUcbknbTmHKPgriipCcxKr3WoLutOQfmnH+DCzt+aH': 2,
         'A39X2AdASSSRD0/g/Vq93r/zcmRaPTm/AOmZLRUPk4JT': 2,
         'AwGkuo9bw0c7CdtuADRYgBj5xuXvWYvuccjOoNSSihCs': 1,
         'ApZQs9+L8HFZj+pSHiJb2BEjXEWa3d9KJd39pa05iqSY': 1})

In [157]:
blocks = Blocks(db=db1)

In [158]:
blocks[10].transactions

[]

In [207]:
[t for t in blocks[10].transactions if t.transaction.signatures[0].signature.data.startswith("aF4")]

[{'_id': ObjectId('5cdd4d7985699a37350740fe'),
  'transaction': {'id': {'type': 'SHA256',
    'data': '+8YKoBO2WH48YIYy0Zlto0+ng1B2XCWDrww698TkBg0='},
   'inputs': [{'id': {'type': 'SHA256',
      'data': 'YRI/oYLChiXWY5r2TXmEZLbHPvjCONCCGScusb2ki88='},
     'outputId': 1,
     'signatureId': 0}],
   'outputs': [{'address': {'data': 'N2DJgbU3v8tbbzaDdBEyn5fPnLZVzAFC6E'},
     'value': {'value': '20150'}},
    {'address': {'data': 'N3kfoQECGGnwsPp5uDqKJXoxjNRdpw8ML7'},
     'value': {'value': '20150'}}],
   'signatures': [{'signature': {'type': 'SHA256',
      'data': 'aF4EAzq9P2dI0NkG0ymwuwbxVHKnwPYZ+uMSx4xNp1KJkcGx3C38oszbDshpvmBrpGggTW/O9KnjjDOtSK52ng=='},
     'keyPub': {'type': 'ECP256K1',
      'rawData': 'A/onLQq4MiXW88WcElvVwuglgo26edpKwnEGverU8qpL'}}]},
  'isCoinbase': False,
  'blockId': {'type': 'SHA256',
   'data': 'qRmznA1gT236hM8PZq+IuB8xIdNA9ehRP1mvmx+TBYk='}}]

In [50]:
[
    (ob.get("branchPath"), ob["block"]["header"]["height"])
    for ob in db2.blocks.find({"branchPath.branchIds": {"$ne": [0]}})
]

[]

In [51]:
[
    (ob.get("branchPath"), ob["block"]["header"]["height"])
    for ob in db0.blocks.find({"branchPath.branchIds": {"$ne": [0]}})
]

[]

In [52]:
heights = [
    [ob["block"]["header"]["height"] for ob in db.blocks.find()]
    for db in (db0, db1, db2)
]

In [53]:
h0 = set(heights[0])
h1 = set(heights[1])
h2 = set(heights[2])

In [56]:
print(h0 - h1, h1 - h0)
print(h0 - h2, h2 - h0)
print(max(h0), len(h0))

set() set()
set() set()
608721 290


In [210]:
Transactions("WXKuOLzlIs8NmdTdf3CVbWFeXLugCl+iV2USK00zGDs=").block.branchPath

[]

In [127]:
Block("6Kp8smlbNZsqhkH7gVgKvE/CnZQhI7XfAkjfLB6501E=")

{'_id': ObjectId('5cdc0c3485699a1b8f49a9fe'),
 'block': {'header': {'id': {'type': 'SHA256',
    'data': '6Kp8smlbNZsqhkH7gVgKvE/CnZQhI7XfAkjfLB6501E='},
   'timestamp': {'data': 1557924916},
   'previousBlockHash': {'type': 'SHA256',
    'data': '8vvhrXnrhobPqyorpo2u6d6bW41JackRcPsRXz3lrOw='},
   'author': {'signature': {'type': 'SHA256',
     'data': 'Vga//8WLii/hx9PwY0nw3eN1abViBLtm2ddtgWajS6iXWSpwEzG3i+08rqfJeEANAnGOHY4rgI/U+vO73bvFNg=='},
    'keyPub': {'type': 'ECP256K1',
     'rawData': 'AyMKGnZhazXAoXb0SJIqi4Ho+lk6bw0UttE34PDtDbya'}},
   'height': 624528}},
 'branch': 'UNVERIFIED',
 'branchPath': {'branchIds': [1, 0], 'blockNumbers': [255, 0]}}

In [38]:
Transactions("0eLNKWSfT51YkpVC6CZRWY9BOX2AUZIZseQxvFssCWs=", db=db1, pool=False).block.branchPath

[{'branchIds': [0], 'blockNumbers': [5]}]

In [147]:
Transactions("Bpgw/tkmuJFEYIqY7EYnfGpRCPa2uh3YAWRQiKplqPY=").block.branchPath

[{'branchIds': [0], 'blockNumbers': [274]}]

In [98]:
Transactions.by_input(transaction_id="+YVQHU/6XaJlPzBJZbbGRqT71HVfJmHtkFnTtc1c+iM=", output_id=1, pool=False)

[{'_id': ObjectId('5ce412b285699a0ee568a07e'),
  'transaction': {'id': {'type': 'SHA256',
    'data': '9WDXjvIpuslIg8Iz5QMWJunPJNVpGDhxfLq/B7ORK48='},
   'inputs': [{'id': {'type': 'SHA256',
      'data': '+YVQHU/6XaJlPzBJZbbGRqT71HVfJmHtkFnTtc1c+iM='},
     'outputId': 1,
     'signatureId': 0},
    {'id': {'type': 'SHA256',
      'data': 'njxTwrxbnM0ijNCy996/Mr9u29WFv70vUQDqGW/VyJo='},
     'outputId': 0,
     'signatureId': 0},
    {'id': {'type': 'SHA256',
      'data': 'OJiEsehwGfnAVH8N4xKaoDtLiCLc+Rq2QlC0sgHRu20='},
     'outputId': 0,
     'signatureId': 0},
    {'id': {'type': 'SHA256',
      'data': 'phFHCnfT0Re92PFaqAjcxjUYoSWbFMc/AMaXeQIZbX4='},
     'outputId': 1,
     'signatureId': 0}],
   'outputs': [{'address': {'data': 'NHW276q4ZQvaDTDGWLvnrfrnVsUY18Esjm'},
     'value': {'value': '495759'}},
    {'address': {'data': 'N3kfoQECGGnwsPp5uDqKJXoxjNRdpw8ML7'},
     'value': {'value': '495760'}}],
   'signatures': [{'signature': {'type': 'SHA256',
      'data': 'Wdr16sp13P65

In [7]:
Transactions.by_input("TLpkCnH0MC5MaRz0dNkUdu7ZATCVbzFdbu3u8e9LOqg=", output_id=0)

[{'_id': ObjectId('5ce2966e85699a2c081d83b7'),
  'transaction': {'id': {'type': 'SHA256',
    'data': '84jnlinty7WczW136n9TJzq91YdtDng6AwHUtqhgt7Y='},
   'inputs': [{'id': {'type': 'SHA256',
      'data': 'TLpkCnH0MC5MaRz0dNkUdu7ZATCVbzFdbu3u8e9LOqg='},
     'outputId': 0,
     'signatureId': 0},
    {'id': {'type': 'SHA256',
      'data': 'RI3FHB4DK8RpWza9AucKVXTPlyaVF0lpx7S+e0k6Mbo='},
     'outputId': 1,
     'signatureId': 0}],
   'outputs': [{'address': {'data': 'NHW276q4ZQvaDTDGWLvnrfrnVsUY18Esjm'},
     'value': {'value': '458775'}},
    {'address': {'data': 'N2DJgbU3v8tbbzaDdBEyn5fPnLZVzAFC6E'},
     'value': {'value': '458775'}}],
   'signatures': [{'signature': {'type': 'SHA256',
      'data': '1eTQxiQzp/YPwDdzMdWZUd6PxKNzgfwMUGwzN+RneyLs3W3XpGvxsgO2D5ni1NAL18tSDxu12LYUSFFt2pcYiQ=='},
     'keyPub': {'type': 'ECP256K1',
      'rawData': 'A7AOi95K1rfyo5tqawwc1chVVl3IXAu2ZxL3MYLGmWQ6'}},
    {'signature': {'type': 'SHA256',
      'data': 'Bg9CMpleVg3ec/Q1oCrTs46m8pNw+P2mP5Dkuf7

In [22]:
Block("KhlcPujzAs58+e9LT/2XeNueJEbZEvT4efAAtnLD6Uk=")

{'_id': ObjectId('5cc9969e85699a12a228294d'),
 'block': {'header': {'id': {'type': 'SHA256',
    'data': 'KhlcPujzAs58+e9LT/2XeNueJEbZEvT4efAAtnLD6Uk='},
   'timestamp': {'data': 1556715166},
   'previousBlockHash': {'type': 'SHA256',
    'data': 'SWAo24vprvsuLKwkpuKHIlyfxB36QEPh0eUV1CnsGpA='},
   'author': {'signature': {'type': 'SHA256',
     'data': 'MQPzbioZt9p9ie0p43cq3/qMiVGaN54coHIPjqQnZ2nQHF/D5xlpQqn+B1F1cjhW6c1WYleHDDPQ7ChgsPB/9g=='},
    'keyPub': {'type': 'ECP256K1',
     'rawData': 'Awqh/GaK+zXAydO8h4sDf4apFNmW8SFaIqRqXioYDXy8'}},
   'height': 382578}},
 'branch': 'UNVERIFIED',
 'branchPath': {'branchIds': [0], 'blockNumbers': [31]}}